# Credit card data

### About the dataset


This dataset can be found in the following [link](https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud)

This dataset has information about the credit card fraud



In [13]:
# Credit card classification

# import statement
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder


def load_data(file_name):
    # load dataset
    dataset = pd.read_csv(file_name)
    
    # replace the cells with single and extra spaces with a Nan value
    dataset.replace(r'^\s*$', np.nan, regex=True, inplace = True)
    
    # convert the dataset to dataframe
    dataset = pd.DataFrame(dataset)
    
    return dataset

dataset = load_data('creditcard.csv')

# print the shape of the dataset
print(dataset.shape)

# print the datatypes of the dataset
print(dataset.dtypes)

# head and tail of the dataset
print(dataset.head(2))
print(dataset.tail(2))


(284807, 31)
Time      float64
V1        float64
V2        float64
V3        float64
V4        float64
V5        float64
V6        float64
V7        float64
V8        float64
V9        float64
V10       float64
V11       float64
V12       float64
V13       float64
V14       float64
V15       float64
V16       float64
V17       float64
V18       float64
V19       float64
V20       float64
V21       float64
V22       float64
V23       float64
V24       float64
V25       float64
V26       float64
V27       float64
V28       float64
Amount    float64
Class       int64
dtype: object
   Time        V1        V2        V3        V4        V5        V6        V7  \
0   0.0 -1.359807 -0.072781  2.536347  1.378155 -0.338321  0.462388  0.239599   
1   0.0  1.191857  0.266151  0.166480  0.448154  0.060018 -0.082361 -0.078803   

         V8        V9  ...       V21       V22       V23       V24       V25  \
0  0.098698  0.363787  ... -0.018307  0.277838 -0.110474  0.066928  0.128539   
1  0.085102

## Check for the missing values

we will check for the missing values in the dataset.

- If there are any missing values, we will replace them with the mean of the column

In [14]:
dataset.isnull().sum()  # There are no missing values in the dataset

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

## Scaling the data 

We will scale the data using the `minmax_scaler` from `sklearn.preprocessing`

In [15]:
def scale_data(dataset):
    # get the column names of each column without the last column
    columns = dataset.columns[:-1]
    # for each column, get the max and min values
    for column in columns:
        max_value = dataset[column].max()
        min_value = dataset[column].min()
        dataset[column] = (dataset[column] - min_value)/(max_value - min_value)
        
    return dataset

dataset = scale_data(dataset)

# print the head of the dataset
print(dataset.head(2))


   Time        V1        V2        V3        V4        V5        V6        V7  \
0   0.0  0.935192  0.766490  0.881365  0.313023  0.763439  0.267669  0.266815   
1   0.0  0.978542  0.770067  0.840298  0.271796  0.766120  0.262192  0.264875   

         V8        V9  ...       V21       V22       V23       V24       V25  \
0  0.786444  0.475312  ...  0.561184  0.522992  0.663793  0.391253  0.585122   
1  0.786298  0.453981  ...  0.557840  0.480237  0.666938  0.336440  0.587290   

        V26       V27       V28    Amount  Class  
0  0.394557  0.418976  0.312697  0.005824      0  
1  0.446013  0.416345  0.313423  0.000105      0  

[2 rows x 31 columns]


## Divide the data into dependent and independent variables

We will divide the data into dependent and independent variables

- The dependent variable is `Class`
- The independent variables are the rest of the columns

In [16]:
# function to divide the dataset into dependent and independent variables
def divide_dataset(dataset):
    # divide the dataset into x and y
    dataset_columns_length = len(dataset.columns)
    print(dataset_columns_length)

    x = dataset.iloc[:, 0:(dataset_columns_length-1)].values
    y = dataset.iloc[:, (dataset_columns_length-1)].values
    
    return x, y

x, y = divide_dataset(dataset)

# print the shape of x and y
print(x.shape)
print(y.shape)
print(x[0])
print(y[0])

31
(284807, 30)
(284807,)
[0.         0.93519234 0.76649042 0.8813649  0.31302266 0.76343873
 0.26766864 0.26681518 0.7864442  0.47531173 0.51060048 0.25248432
 0.68090763 0.3715906  0.63559053 0.4460837  0.43439239 0.73717255
 0.65506586 0.59486323 0.58294223 0.56118439 0.52299212 0.66379298
 0.39125268 0.58512179 0.39455679 0.41897614 0.31269663 0.00582379]
0


## Split the data into train and test sets

We will split the data into train and test sets using the `train_test_split` from `sklearn.model_selection`

- The train set will have 70% of the data.

In [17]:
# import train_test_split
from sklearn.model_selection import train_test_split

# funciton to split the dataset into training and testing set
def split_dataset(x, y):
    # split the dataset into training and testing set
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3)
    
    return x_train, x_test, y_train, y_test

x_train, x_test, y_train, y_test = split_dataset(x, y)

# print the shape of x_train, x_test, y_train, y_test
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)
print(x_train.dtype)
print(x_test.dtype)
print(y_train.dtype)
print(y_test.dtype)

(199364, 30)
(85443, 30)
(199364,)
(85443,)
float64
float64
int64
int64


### Model

- import the custom model from `custom_logistic_regression.py`

In [18]:
from custom_logistic_regression import CustomLogisticRegression

# create an object of CustomLogisticRegression
classifier = CustomLogisticRegression(learning_rate=0.01, num_iterations=20000, early_stopping_threshold=0.01, verbose=True, num_features=20)

classifier.fit(x_train, y_train)

# predict the values
y_pred = classifier.predict(x_test)

# print the accuracy
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

loss: 0.6749623896181135 	


KeyboardInterrupt: 